In [42]:
import numpy as np
from scipy.sparse import diags, hstack, vstack, block_diag
import math

def dft_list(N, pow5):
    length = N << 3
    k = np.array([i for i in range(N)])
    return np.exp(2j * np.pi * pow5[k] / length)

def pow5_list(N):
    length = N << 3
    pow5 = np.arange(N)
    pow5[0] = 1
    for i in range(1, N):
        pow5[i] = 5 * pow5[i - 1]
        pow5[i] = pow5[i] % (length)
        # print(pow5[i])

    return pow5

N = 8
slot = N // 2

log2_value = math.log2(slot) 
depth = int(log2_value)

V = [None] * depth

for i in range(depth):
    block_num = 1 << i
    diag_length = slot // (1 << (i + 1))

    e_diag = np.array([1+0j for length in range(diag_length)])
    e = diags(e_diag)

    pow5 = pow5_list(diag_length)
    omiga_diag = np.array(dft_list(diag_length, pow5))
    omiga = diags(omiga_diag)

    combined_matrix0 = hstack([e, omiga])
    combined_matrix1 = hstack([e, -omiga])
    block = vstack([combined_matrix0, combined_matrix1])
    
    matrix = block_diag([block] * block_num)

    V[i] = matrix

    # print(matrix.toarray())

for i in range(1, depth):
    V[i] =  V[i - 1] @ V[i]

print(V[depth - 1].toarray())

[[ 1.        +0.j          0.70710678+0.70710678j  0.92387953+0.38268343j
   0.38268343+0.92387953j]
 [ 1.        +0.j         -0.70710678-0.70710678j -0.38268343+0.92387953j
   0.92387953-0.38268343j]
 [ 1.        +0.j          0.70710678+0.70710678j -0.92387953-0.38268343j
  -0.38268343-0.92387953j]
 [ 1.        +0.j         -0.70710678-0.70710678j  0.38268343-0.92387953j
  -0.92387953+0.38268343j]]


In [46]:
class ComplexNumberFileManager:
    """
    一个用于保存和加载复数数据的类。
    """

    def __init__(self, filename):
        self.filename = filename

    def save_to_file(self, complex_list):
        """
        将复数列表保存到文件中，实部和虚部分开保存。

        参数:
        complex_list (list): 要保存的复数列表。
        """
        try:
            with open(self.filename, "w") as file:
                for complex_num in complex_list:
                    real_part = complex_num.real
                    imag_part = complex_num.imag
                    file.write(f"{real_part} {imag_part}\n")
            print(f"复数列表已成功保存到文件 {self.filename}")
        except Exception as e:
            print(f"保存复数列表到文件时出错: {e}")

    def load_from_file(self):
        """
        从文件中加载复数列表。

        返回:
        list: 加载的复数列表。
        """
        complex_list = []
        try:
            with open(self.filename, "r") as file:
                for line in file:
                    real_part, imag_part = map(float, line.split())
                    complex_num = complex(real_part, imag_part)
                    complex_list.append(complex_num)
            print(f"复数列表已成功从文件 {self.filename} 加载")
        except Exception as e:
            print(f"从文件加载复数列表时出错: {e}")

        return complex_list

# 示例使用
# filename = "complex_numbers.txt"
# manager = ComplexNumberFileManager(filename)

# 保存复数列表
# complex_list = [1+1j, 2+2j, 3+3j, 4+4j]
# manager.save_to_file(complex_list)

# 从文件加载复数列表
# loaded_complex_list = manager.load_from_file()
# print(f"加载的复数列表：{loaded_complex_list}")


In [47]:
import numpy as np
from scipy.sparse import diags, hstack, vstack, block_diag
import math


def dft_list(N, pow5):
    length = N << 3
    k = np.array([i for i in range(N)])
    return np.exp(2j * np.pi * pow5[k] / length)

def pow5_list(N):
    length = N << 3
    pow5 = np.arange(N)
    pow5[0] = 1
    for i in range(1, N):
        pow5[i] = 5 * pow5[i - 1]
        pow5[i] = pow5[i] % (length)
        # print(pow5[i])

    return pow5

N = 1 << 12
slot = N // 2

log2_value = math.log2(slot) 
depth = int(log2_value)

V = [None] * depth

omiga_lists = [None] * depth

for i in range(depth):
    block_num = 1 << i
    diag_length = slot // (1 << (i + 1))

    e_diag = np.array([1+0j for length in range(diag_length)])
    e = diags(e_diag)

    pow5 = pow5_list(diag_length)
    omiga_list = dft_list(diag_length, pow5)
    omiga_lists[i] = omiga_list

    omiga_diag = np.array(omiga_list)
    omiga = diags(omiga_diag)

    combined_matrix0 = hstack([e, omiga])
    combined_matrix1 = hstack([e, -omiga])
    block = vstack([combined_matrix0, combined_matrix1])
    
    matrix = block_diag([block] * block_num)

    V[i] = matrix

    # print(matrix.toarray())

# for i in range(1, depth):
#     V[i] =  V[i - 1] @ V[i]

# print(V[depth - 1].toarray())

file_path = '/home/syt/data/phantom/file/omiga/'
for i in range(depth):
    filename = file_path + "omiga_list_depth_" + str(i) + "_.txt"
    manager = ComplexNumberFileManager(filename)
    manager.save_to_file(omiga_lists[i])


复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_0_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_1_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_2_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_3_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_4_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_5_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_6_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_7_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_8_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_9_.txt
复数列表已成功保存到文件 /home/syt/data/phantom/file/omiga/omiga_list_depth_10_.txt


In [56]:
import numpy as np

def compare_complex_arrays_with_precision(arr1, arr2, tol=1e-5):
    """
    比较两个复数数组，相同则返回True，不同则输出不同的位置。
    考虑浮点精度。

    参数:
    arr1, arr2: 两个要比较的复数数组
    tol: 浮点精度容差

    返回:
    bool: 如果相同返回True，否则返回False并输出不同的位置
    """
    # if arr1.shape != arr2.shape:
    #     print("数组形状不同")
    #     return False

    # 比较两个数组，考虑浮点精度
    comparison = np.isclose(arr1, arr2, atol=tol)
    if np.all(comparison):
        return True
    else:
        # 找出不同的位置
        different_positions = np.where(comparison == False)
        print("不同的位置：", different_positions)
        return False

loaded_complex_list0 = []
loaded_complex_list1 = []

file_path0 = '/home/syt/data/phantom/file/omiga/python/'
file_path1 = '/home/syt/data/phantom/file/omiga/cpp/'

for i in range(depth):

    filename0 = file_path0 + "omiga_list_depth_" + str(i) + "_.txt"
    filename1 = file_path1 + "omiga_list_depth_" + str(i) + "_.txt"

    manager0 = ComplexNumberFileManager(filename0)
    manager1 = ComplexNumberFileManager(filename1)

    loaded_complex_list0 = manager0.load_from_file() 
    loaded_complex_list1 = manager1.load_from_file() 

    # print(loaded_complex_list0.shape)
    # print(loaded_complex_list1.shape)

    result = compare_complex_arrays_with_precision(loaded_complex_list0, loaded_complex_list1)
    print(result)


复数列表已成功从文件 /home/syt/data/phantom/file/omiga/python/omiga_list_depth_0_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/cpp/omiga_list_depth_0_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/python/omiga_list_depth_1_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/cpp/omiga_list_depth_1_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/python/omiga_list_depth_2_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/cpp/omiga_list_depth_2_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/python/omiga_list_depth_3_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/cpp/omiga_list_depth_3_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/python/omiga_list_depth_4_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/cpp/omiga_list_depth_4_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/python/omiga_list_depth_5_.txt 加载
复数列表已成功从文件 /home/syt/data/phantom/file/omiga/cpp/omiga_list_depth_5_.txt 加载
True
复数列表已成功从文件 /home/syt/data/phantom/file/o